In [1]:
import sys
from pathlib import Path
from datetime import datetime
import shutil
from pmpnn import main as run_protein_mpnn
from extract_best_seq import parse_and_extract
from IPython.display import display, clear_output
from create_empty_msas import main as create_empty_msas
from debug_openfold import run_prediction as run_openfold
from add_linker import convert_fasta

[2024-04-30 00:26:58,349] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
PREFIX = "looping"

current_date = datetime.now()
ddmmyy_prefix = current_date.strftime("%d%b%y")

OUTPUT_DIR = Path("/nfs/homedirs/hetzell/code/protein_design/example_outputs") / f"{ddmmyy_prefix}_{PREFIX}"

if not OUTPUT_DIR.exists():
    OUTPUT_DIR.mkdir(parents=True)

print(OUTPUT_DIR)

/nfs/homedirs/hetzell/code/protein_design/example_outputs/30Apr24_looping


In [5]:
OUTPUT_DIR = Path("/nfs/homedirs/hetzell/code/protein_design/example_outputs") / f"29Apr24_looping"

In [6]:
C8_designs = Path("/nfs/homedirs/hetzell/code/protein_project/protein-backbone-MCTS/outputs/C_small_chunks")
INPUT_PDB = C8_designs / "Subfolder_1" 
OUTPUT_PDB = OUTPUT_DIR/"pdbs"
# Ensure the output directory exists
if not OUTPUT_PDB.exists():
    OUTPUT_PDB.mkdir(parents=True, exist_ok=False)

# Move all PDB files to OUTPUT_DIR
for pdb in INPUT_PDB.iterdir():
    if pdb.is_file() and pdb.suffix.lower() == '.pdb':  # Ensure it's a file and has the .pdb extension
        target_path = OUTPUT_PDB/ pdb.name
        if target_path.exists():
            continue
        try:
            shutil.copy(str(pdb), str(target_path))
        except Exception as e:
            print(f"Failed to move {pdb} to {target_path}: {e}")


In [8]:
run_protein_mpnn(str(OUTPUT_PDB), str(OUTPUT_DIR/'protein_mpnn'))

----------------------------------------
chain_id_jsonl is NOT loaded
----------------------------------------
fixed_positions_jsonl is NOT loaded
----------------------------------------
pssm_jsonl is NOT loaded
----------------------------------------
omit_AA_jsonl is NOT loaded
----------------------------------------
bias_AA_jsonl is NOT loaded
----------------------------------------
bias by residue dictionary is not loaded, or not provided
----------------------------------------
discarded {'bad_chars': 0, 'too_long': 0, 'bad_seq_length': 0}
----------------------------------------
Number of edges: 48
Training noise level: 0.2A
Generating sequences for: build_8_0.2203_1.0_34_2.55_0.016683
10 sequences of length 472 generated in 1.6156 seconds
Generating sequences for: build_8_0.2105_1.0_20_2.11_0.004154
10 sequences of length 608 generated in 1.9614 seconds
Generating sequences for: build_8_0.2203_1.0_12_2.41_0.003378
10 sequences of length 472 generated in 1.5147 seconds
Generat

In [7]:
FASTA_DIR = OUTPUT_DIR/ "protein_mpnn" / "seqs"
OUTPUT_SEQS = FASTA_DIR.parent / "seqs_best"
fasta_files = [f for f in FASTA_DIR.iterdir() if f.suffix in [".fasta", ".fa"]]

for i, f in enumerate(fasta_files):
    parse_and_extract(f, OUTPUT_SEQS, overwrite=True, n_seqs=3)
    print(f"Sequences {i+1:5}/{len(fasta_files):5} done.")
    clear_output(wait=True)

Sequences   100/  100 done.


In [8]:
fasta_files = [f for f in (OUTPUT_SEQS/"trimer").iterdir() if f.suffix in [".fasta", ".fa"]]
OUTPUT_LINKER = OUTPUT_SEQS / "trimer_linker"

OUTPUT_LINKER.mkdir(parents=True, exist_ok=True)

for i, f in enumerate(fasta_files):
    o = OUTPUT_LINKER / f"{f.stem}_linker.fasta"
    convert_fasta(f, o)
    print(f"Sequences {i+1:5}/{len(fasta_files):5} done.\n")
    clear_output(wait=True)

Sequences   300/  300 done.



In [20]:
setting = 'trimer_linker'

OUTPUT_OPENFOLD = OUTPUT_DIR / f"openfold_{setting}"
FASTA_DIR = OUTPUT_LINKER

create_empty_msas(str(FASTA_DIR), str(OUTPUT_OPENFOLD / "alignments"))

In [21]:
FASTA_DIR

PosixPath('/nfs/homedirs/hetzell/code/protein_design/example_outputs/29Apr24_looping/protein_mpnn/seqs_best/trimer_linker')

In [ ]:
# # Only test on a subset of sequences
# _FASTA_DIR = FASTA_DIR.parent / "trimer_linker_test"

run_openfold(
    fasta_dir=str(FASTA_DIR),
    use_precomputed_alignments=str(OUTPUT_OPENFOLD / "alignments"),
    config_preset="model_1",
    model_device="cuda:0",
    output_dir=str(OUTPUT_OPENFOLD),
)

In [17]:
setting = "homomer"

OUTPUT_OPENFOLD = OUTPUT_DIR / f"openfold_{setting}"
# Only test on a subset of sequences
_FASTA_DIR = OUTPUT_DIR / "protein_mpnn" / "seqs_best" / "homomer"

create_empty_msas(str(_FASTA_DIR), str(OUTPUT_OPENFOLD / "alignments"))

In [19]:
# Only test on a subset of sequences
_FASTA_DIR = FASTA_DIR.parent / "homomer_test"

run_openfold(
    fasta_dir=str(_FASTA_DIR),
    use_precomputed_alignments=str(OUTPUT_OPENFOLD / "alignments"),
    config_preset="model_1_multimer_v3",
    model_device="cuda:0",
    output_dir=str(OUTPUT_OPENFOLD),
)

INFO:/nfs/staff-ssd/hetzell/code/protein_design/modules/openfold/openfold/utils/script_utils.py:Successfully loaded JAX parameters at modules/openfold/openfold/resources/params/params_model_1_multimer_v3.npz...
INFO:/nfs/staff-ssd/hetzell/code/protein_design/modules/openfold/run_pretrained_openfold.py:Skipping build_8_0.2_0.94_11_2.03_0.002697_0__A-B-C-D-E-F-G-H_model_1_multimer_v3 as it already exists...
